# Correlating Returns

In [1]:
import os
import pandas as pd
from datetime import datetime, timedelta
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from newsapi.newsapi_client import NewsApiClient
from nltk.sentiment.vader import SentimentIntensityAnalyzer

## Load API Keys from Environment Variables

In [2]:
# Load .env enviroment variables
load_dotenv()

# Set News API Key
newsapi = NewsApiClient(api_key=os.environ["NEWS_API_KEY"])
#newsapi = NewsApiClient(api_key=os.getenv("NEWS_API_KEY"))

# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_API_SECRET_KEY")

api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')

print(f"News API Key type: {type(newsapi)}")
print(f"News ALPACA Key type: {type(alpaca_api_key)}")
print(f"News ALPACA SECRET Key type: {type(alpaca_secret_key)}")

News API Key type: <class 'newsapi.newsapi_client.NewsApiClient'>
News ALPACA Key type: <class 'str'>
News ALPACA SECRET Key type: <class 'str'>


## Get AAPL Returns for Past Month

In [3]:
# Set the ticker
ticker = "AAPL"

# Set timeframe to '1D'
timeframe = "1D"

# Set current date and the date from one month ago using the ISO format
current_date = pd.Timestamp(datetime.now(), tz="America/New_York").isoformat()
past_date = pd.Timestamp(datetime.now()- timedelta(30), tz="America/New_York").isoformat()

# Get 4 weeks worth of historical data for AAPL
df = api.get_barset(ticker,
                    timeframe,
                    limit=None,
                    start=past_date,
                    end=current_date,
                    after=None,
                    until=None).df

# Display data
df.head()

AAPL                                    
                             open    high       low   close    volume
time                                                                 
2021-07-06 00:00:00-04:00  140.10  143.15  140.0700  142.02  97272256
2021-07-07 00:00:00-04:00  143.54  144.89  142.6600  144.59  93223232
2021-07-08 00:00:00-04:00  141.57  144.06  140.6650  143.32  94133450
2021-07-09 00:00:00-04:00  142.73  145.65  142.6522  145.11  89995907
2021-07-12 00:00:00-04:00  146.21  146.32  144.0000  144.50  65946147

In [4]:
# Drop Outer Table Level
df = df.droplevel(axis=1, level=0)

# Use the drop function to drop extra columns
df = df.drop(columns=["open", "high", "low", "volume"])

# Since this is daily data, we can keep only the date (remove the time) component of the data
df.index = df.index.date

# Display sample data
df.head()

,close
2021-07-06,142.02
2021-07-07,144.59
2021-07-08,143.32
2021-07-09,145.11
2021-07-12,144.50


In [5]:
# Use the `pct_change` function to calculate daily returns of AAPL
aapl_returns = df.pct_change().dropna()

# Display sample data
aapl_returns.head()

,close
2021-07-07,0.018096
2021-07-08,-0.008783
2021-07-09,0.012490
2021-07-12,-0.004204
2021-07-13,0.008166


In [6]:
# Use newsapi client to get most relevant 20 headlines per day in the past month
def get_headlines(keyword):
    all_headlines = []
    
    all_dates = []    
    
    date = datetime.strptime(current_date[:10], "%Y-%m-%d")
    
    end_date = datetime.strptime(past_date[:10], "%Y-%m-%d")
    
    print(f"Fetching news about '{keyword}'")
    
    print("*" * 30)
    
    while date > end_date:
        
        print(f"retrieving news from: {date}")
        
        articles = newsapi.get_everything(q=keyword,
                                          from_param=str(date)[:10],
                                          to=str(date)[:10],
                                          language="en",
                                          sort_by="relevancy",
                                          page=1)
        headlines = []
        
        for i in range(0, len(articles["articles"])):
            
            headlines.append(articles["articles"][i]["title"])
            
        all_headlines.append(headlines)
        
        all_dates.append(date)
        
        date = date - timedelta(days=1)
        
    return all_headlines, all_dates

Note: Be aware that running the 3 requests below will only work once within a 24 hour period due to the request limits imposed by the API provider.

In [7]:
# Get first topic
apple_news = get_headlines('apple')

Fetching news about 'apple'
******************************
retrieving news from: 2021-08-02 00:00:00
retrieving news from: 2021-08-01 00:00:00
retrieving news from: 2021-07-31 00:00:00
retrieving news from: 2021-07-30 00:00:00
retrieving news from: 2021-07-29 00:00:00
retrieving news from: 2021-07-28 00:00:00
retrieving news from: 2021-07-27 00:00:00
retrieving news from: 2021-07-26 00:00:00
retrieving news from: 2021-07-25 00:00:00
retrieving news from: 2021-07-24 00:00:00
retrieving news from: 2021-07-23 00:00:00
retrieving news from: 2021-07-22 00:00:00
retrieving news from: 2021-07-21 00:00:00
retrieving news from: 2021-07-20 00:00:00
retrieving news from: 2021-07-19 00:00:00
retrieving news from: 2021-07-18 00:00:00
retrieving news from: 2021-07-17 00:00:00
retrieving news from: 2021-07-16 00:00:00
retrieving news from: 2021-07-15 00:00:00
retrieving news from: 2021-07-14 00:00:00
retrieving news from: 2021-07-13 00:00:00
retrieving news from: 2021-07-12 00:00:00
retrieving news f

In [8]:
# Get second topic
trade_news = get_headlines('trade')

Fetching news about 'trade'
******************************
retrieving news from: 2021-08-02 00:00:00
retrieving news from: 2021-08-01 00:00:00
retrieving news from: 2021-07-31 00:00:00
retrieving news from: 2021-07-30 00:00:00
retrieving news from: 2021-07-29 00:00:00
retrieving news from: 2021-07-28 00:00:00
retrieving news from: 2021-07-27 00:00:00
retrieving news from: 2021-07-26 00:00:00
retrieving news from: 2021-07-25 00:00:00
retrieving news from: 2021-07-24 00:00:00
retrieving news from: 2021-07-23 00:00:00
retrieving news from: 2021-07-22 00:00:00
retrieving news from: 2021-07-21 00:00:00
retrieving news from: 2021-07-20 00:00:00
retrieving news from: 2021-07-19 00:00:00
retrieving news from: 2021-07-18 00:00:00
retrieving news from: 2021-07-17 00:00:00
retrieving news from: 2021-07-16 00:00:00
retrieving news from: 2021-07-15 00:00:00
retrieving news from: 2021-07-14 00:00:00
retrieving news from: 2021-07-13 00:00:00
retrieving news from: 2021-07-12 00:00:00
retrieving news f

In [9]:
# Get third topic
economy_news = get_headlines('economy')

Fetching news about 'economy'
******************************
retrieving news from: 2021-08-02 00:00:00
retrieving news from: 2021-08-01 00:00:00
retrieving news from: 2021-07-31 00:00:00
retrieving news from: 2021-07-30 00:00:00
retrieving news from: 2021-07-29 00:00:00
retrieving news from: 2021-07-28 00:00:00
retrieving news from: 2021-07-27 00:00:00
retrieving news from: 2021-07-26 00:00:00
retrieving news from: 2021-07-25 00:00:00
retrieving news from: 2021-07-24 00:00:00
retrieving news from: 2021-07-23 00:00:00
retrieving news from: 2021-07-22 00:00:00
retrieving news from: 2021-07-21 00:00:00
retrieving news from: 2021-07-20 00:00:00
retrieving news from: 2021-07-19 00:00:00
retrieving news from: 2021-07-18 00:00:00
retrieving news from: 2021-07-17 00:00:00
retrieving news from: 2021-07-16 00:00:00
retrieving news from: 2021-07-15 00:00:00
retrieving news from: 2021-07-14 00:00:00
retrieving news from: 2021-07-13 00:00:00
retrieving news from: 2021-07-12 00:00:00
retrieving news

In [10]:
# Instantiate SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [11]:
# Create function that computes average compound sentiment of headlines for each day
def headline_sentiment_summarizer_avg(headlines):
    sentiment = []
    for day in headlines:
        day_score = []
        for h in day:
            if h == None:
                continue
            else:
                day_score.append(analyzer.polarity_scores(h)["compound"])
        sentiment.append(sum(day_score) / len(day_score))
    return sentiment

In [12]:
# Apple sentiment analysis - Compound score
apple_headlines = apple_news[0][1:]

apple_headlines[0]

apple_sentiment= headline_sentiment_summarizer_avg(apple_headlines)

apple_sentiment[0]

0.06222500000000001

In [13]:
# Trade sentiment analysis - Compound score
trade_headlines = trade_news[0][1:]

trade_headlines[0]

trade_sentiment= headline_sentiment_summarizer_avg(trade_headlines)

trade_sentiment[0]

-0.007714999999999991

In [14]:
# Trade sentiment analysis - Compound score
economy_headlines = economy_news[0][1:]

economy_headlines[0]

economy_sentiment= headline_sentiment_summarizer_avg(economy_headlines)

economy_sentiment[0]

-0.01676499999999999

In [22]:
# Combine Sentiment Averages into DataFrame
comb_sents = list(zip(apple_sentiment, trade_sentiment, economy_sentiment))

sentiment_df = pd.DataFrame(comb_sents,
                            columns=["APPLE", "TRADE", "ECONOMY"])

sentiment_df.head()

,APPLE,TRADE,ECONOMY
0,0.062225,-0.007715,-0.016765
1,0.082715,0.089820,-0.018845
2,-0.035465,-0.008445,0.107345
3,0.116940,-0.045290,-0.034840
4,0.046710,-0.016005,-0.030875


In [23]:
# Set the index value of the sentiment averages DataFrame to be the series of dates.
dates = apple_news[1][1:]

sentiment_df["DATES"] = dates

sentiment_df.set_index("DATES", inplace=True)

sentiment_df.head()

,APPLE,TRADE,ECONOMY
DATES,,,
2021-08-01,0.062225,-0.007715,-0.016765
2021-07-31,0.082715,0.089820,-0.018845
2021-07-30,-0.035465,-0.008445,0.107345
2021-07-29,0.116940,-0.045290,-0.034840
2021-07-28,0.046710,-0.016005,-0.030875


In [25]:
sentiment_df.shape

(29, 3)

In [26]:
# Merge with AAPL returns
merged_df = pd.concat([aapl_returns['close'], sentiment_df], axis=1)

merged_final_df = merged_df.dropna()

merged_final_df.head()

,close,APPLE,TRADE,ECONOMY
2021-07-07,0.018096,-0.093715,0.008625,0.035235
2021-07-08,-0.008783,0.040350,0.030665,-0.025300
2021-07-09,0.012490,0.167700,0.096575,-0.049750
2021-07-12,-0.004204,0.030270,-0.011800,0.007635
2021-07-13,0.008166,0.118670,0.091675,0.028325


In [27]:
merged_final_df.shape

(18, 4)

In [31]:
# Correlate the headlines' sentiment to returns
correlation = merged_final_df.corr().style.background_gradient()

correlation

,close,APPLE,TRADE,ECONOMY
close,1.000000,0.054718,-0.031529,0.213257
APPLE,0.054718,1.000000,0.203490,-0.004970
TRADE,-0.031529,0.203490,1.000000,-0.227751
ECONOMY,0.213257,-0.004970,-0.227751,1.000000


In [30]:
 merged_final_df.corr().style.background_gradient()

,close,APPLE,TRADE,ECONOMY
close,1.000000,0.054718,-0.031529,0.213257
APPLE,0.054718,1.000000,0.203490,-0.004970
TRADE,-0.031529,0.203490,1.000000,-0.227751
ECONOMY,0.213257,-0.004970,-0.227751,1.000000
